# VERİ HAZIRLAMA

In [49]:
import pandas as pd
import numpy as np

In [50]:
veriler = pd.read_csv('train.csv', parse_dates=['Tarih'])

In [51]:
veriler.isna().sum()
veriler.describe()
veriler.info()
veriler.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40152 entries, 0 to 40151
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Tarih                   40152 non-null  datetime64[ns]
 1   Dağıtılan Enerji (MWh)  40152 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 627.5 KB


,Tarih,Dağıtılan Enerji (MWh)
0,2018-01-01 00:00:00,1593.944216
1,2018-01-01 01:00:00,1513.933887
2,2018-01-01 02:00:00,1402.612637
3,2018-01-01 03:00:00,1278.527266
4,2018-01-01 04:00:00,1220.697701


In [52]:
veriler = veriler.rename(columns={"Dağıtılan Enerji (MWh)" : "DE"})

In [53]:
veriler.head()

,Tarih,DE
0,2018-01-01 00:00:00,1593.944216
1,2018-01-01 01:00:00,1513.933887
2,2018-01-01 02:00:00,1402.612637
3,2018-01-01 03:00:00,1278.527266
4,2018-01-01 04:00:00,1220.697701


In [54]:
veriler['Gün'] = veriler['Tarih'].dt.dayofweek
veriler["GünSırası"]=veriler["Tarih"].dt.day
veriler['Ay'] = veriler['Tarih'].dt.month
veriler['Yıl'] = veriler['Tarih'].dt.year
veriler['Saat'] = veriler['Tarih'].dt.hour
veriler['Mevsim'] = (veriler['Tarih'].dt.month % 12 + 3) // 3
veriler['Hafta Sonu'] = veriler['Gün'].apply(lambda x: 1 if x in [5, 6] else 0)
veriler['Gece'] = veriler['Saat'].apply(lambda x: 1 if x in [18,19,20,21,22,23,0,1,2,3,4,5] else 0)

In [55]:
def add_quality(Saat):
    if 0<=Saat and Saat <=4: 
        return 0
    elif Saat>20:
        return 0
    elif 11<Saat and Saat<=12:
        return 0
    elif 6<Saat and Saat<=11:
        return 1
    elif 12<Saat and Saat<=14:
        return 1
    elif 5<=Saat and Saat<=6:
        return 2
    elif 14<Saat and Saat<=20:
        return 2
    
veriler["Artmış Mı"] = veriler["Saat"].apply(add_quality)

In [56]:
veriler.head(24000)
#veriler.info()

,Tarih,DE,Gün,GünSırası,Ay,Yıl,Saat,Mevsim,Hafta Sonu,Gece,Artmış Mı
0,2018-01-01 00:00:00,1593.944216,0,1,1,2018,0,1,0,1,0
1,2018-01-01 01:00:00,1513.933887,0,1,1,2018,1,1,0,1,0
2,2018-01-01 02:00:00,1402.612637,0,1,1,2018,2,1,0,1,0
3,2018-01-01 03:00:00,1278.527266,0,1,1,2018,3,1,0,1,0
4,2018-01-01 04:00:00,1220.697701,0,1,1,2018,4,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
23995,2020-09-26 19:00:00,2125.575504,5,26,9,2020,19,4,1,1,2
23996,2020-09-26 20:00:00,2070.230938,5,26,9,2020,20,4,1,1,2
23997,2020-09-26 21:00:00,1960.334625,5,26,9,2020,21,4,1,1,0
23998,2020-09-26 22:00:00,1862.313947,5,26,9,2020,22,4,1,1,0


In [57]:
veriler = veriler.drop('Tarih', axis=1)
veriler= veriler.astype(float)
veri_yedek= veriler
veriler.head(24000)
#veriler.info()

,DE,Gün,GünSırası,Ay,Yıl,Saat,Mevsim,Hafta Sonu,Gece,Artmış Mı
0,1593.944216,0.0,1.0,1.0,2018.0,0.0,1.0,0.0,1.0,0.0
1,1513.933887,0.0,1.0,1.0,2018.0,1.0,1.0,0.0,1.0,0.0
2,1402.612637,0.0,1.0,1.0,2018.0,2.0,1.0,0.0,1.0,0.0
3,1278.527266,0.0,1.0,1.0,2018.0,3.0,1.0,0.0,1.0,0.0
4,1220.697701,0.0,1.0,1.0,2018.0,4.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
23995,2125.575504,5.0,26.0,9.0,2020.0,19.0,4.0,1.0,1.0,2.0
23996,2070.230938,5.0,26.0,9.0,2020.0,20.0,4.0,1.0,1.0,2.0
23997,1960.334625,5.0,26.0,9.0,2020.0,21.0,4.0,1.0,1.0,0.0
23998,1862.313947,5.0,26.0,9.0,2020.0,22.0,4.0,1.0,1.0,0.0


# MODEL EĞİTME

In [58]:
from sklearn.model_selection import train_test_split
def mape(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [59]:
# Bağımlı ve bağımsız değişkenleri ayırma
X = veriler.drop(["DE"], axis=1)
y = veriler["DE"]

In [62]:
# Train-test olarak verileri bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.04, random_state=42)

In [63]:
from sklearn.ensemble import RandomForestRegressor
model1 = RandomForestRegressor(n_estimators=165, random_state=42) 
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)
print('MAPE ', mape(y_test, y_pred1))

MAPE  2.130468187145402


In [14]:
y_pred1.shape

(1607,)

In [15]:
y_pred= pd.DataFrame(y_pred1, columns=['DE'])
y_pred

,DE
0,2081.226144
1,1765.662999
2,1765.922150
3,2678.034531
4,1298.189310
...,...
1602,2389.760708
1603,2593.872378
1604,1334.924546
1605,2158.107718


In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
# verilerimizin model üzerindeki etkisini hesaplıyoruz
perm = PermutationImportance(model1, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

SUBMISSION

In [16]:
sub = pd.read_csv('sample_submission.csv', parse_dates=['Tarih'])
sub

,Tarih,Dağıtılan Enerji (MWh)
0,2022-08-01 00:00:00,0.0
1,2022-08-01 01:00:00,0.0
2,2022-08-01 02:00:00,0.0
3,2022-08-01 03:00:00,0.0
4,2022-08-01 04:00:00,0.0
...,...,...
739,2022-08-31 19:00:00,0.0
740,2022-08-31 20:00:00,0.0
741,2022-08-31 21:00:00,0.0
742,2022-08-31 22:00:00,0.0


In [17]:
sub['Gün'] = sub['Tarih'].dt.dayofweek
sub["GünSırası"]=sub["Tarih"].dt.day
sub['Ay'] = sub['Tarih'].dt.month
sub['Yıl'] = sub['Tarih'].dt.year
sub['Saat'] = sub['Tarih'].dt.hour
sub['Mevsim'] = (sub['Tarih'].dt.month % 12 + 3) // 3
sub['Hafta Sonu'] = sub['Gün'].apply(lambda x: 1 if x in [5, 6] else 0)
sub['Gece'] = sub['Saat'].apply(lambda x: 1 if x in [18,19,20,21,22,23,0,1,2,3,4,5] else 0)

In [18]:
def add_quality(Saat):
    if 0<=Saat and Saat <=4: 
        return 0
    elif Saat>20:
        return 0
    elif 11<Saat and Saat<=12:
        return 0
    elif 6<Saat and Saat<=11:
        return 1
    elif 12<Saat and Saat<=14:
        return 1
    elif 5<=Saat and Saat<=6:
        return 2
    elif 14<Saat and Saat<=20:
        return 2
sub["Artmış Mı"] = sub["Saat"].apply(add_quality)
sub = sub.rename(columns={"Dağıtılan Enerji (MWh)" : "DE"})

In [19]:
sub.head()

,Tarih,DE,Gün,GünSırası,Ay,Yıl,Saat,Mevsim,Hafta Sonu,Gece,Artmış Mı
0,2022-08-01 00:00:00,0.0,0,1,8,2022,0,3,0,1,0
1,2022-08-01 01:00:00,0.0,0,1,8,2022,1,3,0,1,0
2,2022-08-01 02:00:00,0.0,0,1,8,2022,2,3,0,1,0
3,2022-08-01 03:00:00,0.0,0,1,8,2022,3,3,0,1,0
4,2022-08-01 04:00:00,0.0,0,1,8,2022,4,3,0,1,0


In [20]:
sub = sub.drop('Tarih', axis=1)
sub= sub.astype(float)

In [28]:
subY= sub['DE']
subY

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
739    0.0
740    0.0
741    0.0
742    0.0
743    0.0
Name: DE, Length: 744, dtype: float64

In [29]:
subX= sub.drop(['DE'], axis=1, inplace=True)
subX= sub
subX

,Gün,GünSırası,Ay,Yıl,Saat,Mevsim,Hafta Sonu,Gece,Artmış Mı
0,0.0,1.0,8.0,2022.0,0.0,3.0,0.0,1.0,0.0
1,0.0,1.0,8.0,2022.0,1.0,3.0,0.0,1.0,0.0
2,0.0,1.0,8.0,2022.0,2.0,3.0,0.0,1.0,0.0
3,0.0,1.0,8.0,2022.0,3.0,3.0,0.0,1.0,0.0
4,0.0,1.0,8.0,2022.0,4.0,3.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
739,2.0,31.0,8.0,2022.0,19.0,3.0,0.0,1.0,2.0
740,2.0,31.0,8.0,2022.0,20.0,3.0,0.0,1.0,2.0
741,2.0,31.0,8.0,2022.0,21.0,3.0,0.0,1.0,0.0
742,2.0,31.0,8.0,2022.0,22.0,3.0,0.0,1.0,0.0


In [30]:
Y= veriler['DE']
X= veriler.drop(['DE'], axis=1, inplace=True)

KeyError: 'DE'

In [31]:
Y

0        1593.944216
1        1513.933887
2        1402.612637
3        1278.527266
4        1220.697701
            ...     
40147    2492.489498
40148    2479.958245
40149    2485.016884
40150    2396.574255
40151    2285.252844
Name: DE, Length: 40152, dtype: float64

In [32]:
X= veriler
X

,Gün,GünSırası,Ay,Yıl,Saat,Mevsim,Hafta Sonu,Gece,Artmış Mı
0,0.0,1.0,1.0,2018.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,1.0,2018.0,1.0,1.0,0.0,1.0,0.0
2,0.0,1.0,1.0,2018.0,2.0,1.0,0.0,1.0,0.0
3,0.0,1.0,1.0,2018.0,3.0,1.0,0.0,1.0,0.0
4,0.0,1.0,1.0,2018.0,4.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
40147,6.0,31.0,7.0,2022.0,19.0,3.0,1.0,1.0,2.0
40148,6.0,31.0,7.0,2022.0,20.0,3.0,1.0,1.0,2.0
40149,6.0,31.0,7.0,2022.0,21.0,3.0,1.0,1.0,0.0
40150,6.0,31.0,7.0,2022.0,22.0,3.0,1.0,1.0,0.0


In [27]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor(n_estimators=165, random_state=42) 
model2.fit(X, Y)
y_pred2 = model2.predict(subX)
print('MAPE ', mape(subY, y_pred2))

C:\Users\acer\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- DE
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


ValueError: X has 10 features, but RandomForestRegressor is expecting 9 features as input.

In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission["Dağıtılan Enerji (MWh)"] = y_pred2

In [ ]:
submission

In [ ]:
submission.to_csv("submission3.csv", index=False)

In [33]:
from sklearn.ensemble import ExtraTreesRegressor
model3 = ExtraTreesRegressor()
model3.fit(X, Y)
y_pred3 = model3.predict(subX)

In [46]:
import pickle

In [47]:
model_dosyasi="modeldosyasi.pickle"

In [48]:
pickle.dump(model3,open(model_dosyasi, 'wb'))

In [34]:
submission2 = pd.read_csv('sample_submission.csv')

In [35]:
submission2["Dağıtılan Enerji (MWh)"] = y_pred3

In [36]:
submission2

,Tarih,Dağıtılan Enerji (MWh)
0,2022-08-01 00:00:00,2316.159468
1,2022-08-01 01:00:00,2180.876339
2,2022-08-01 02:00:00,1991.651356
3,2022-08-01 03:00:00,1900.047551
4,2022-08-01 04:00:00,1848.619610
...,...,...
739,2022-08-31 19:00:00,2646.496163
740,2022-08-31 20:00:00,2645.440841
741,2022-08-31 21:00:00,2586.444992
742,2022-08-31 22:00:00,2471.396741


In [37]:
submission2.to_csv("submission4.csv", index=False)